# Search - path planning

## First search
1. Start with a grid with a start position S and goal position G.
2. Expand from S into all directions that are available (not blocked off).
3. Keep track of the path length for each cell (g-value).
4. Find the cell with the current shortest path length and expand from there.

In [7]:
import numpy as np

In [2]:
# ----------
#
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space


grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
        [ 0,-1], # go left
        [ 1, 0], # go down
        [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

In [18]:
def search(grid, init, goal, cost, path_length=None, verbose=False):
    if init == goal:
        return ([int(path_length[goal[0], goal[1]]), init[0], init[1]])

    if not isinstance(grid, np.ndarray):
        grid = np.array(grid)
    else:
        grid = grid.copy()

    delta = np.array([
        [-1, 0], # go up
        [ 0,-1], # go left
        [ 1, 0], # go down
        [ 0, 1] # go right
        ])

    # path_length: initialise as nan
    if path_length is None:
        path_length = np.zeros(grid.shape)
        path_length[:, :] = np.nan
        # add initial cell with path length 0
        path_length[init[0], init[1]] = 0

    # mark off the current cell
    grid[init[0], init[1]] = 1

    # check whether neighbouring cells are unmarked (untraveled)
    for d in delta:
        index = init - d
        try:
            # ignore negative indices
            if any(index < 0):
                continue

            cell = grid[index[0], index[1]]
        except(IndexError):
            # ignore out of bounds indices
            continue

        # if cell is unmarked, update the path cost
        if cell == 0:
            path_length[index[0], index[1]] = path_length[init[0], init[1]] + cost

    try:
        # find the next cell: unmarked cell with lowest cost
        unmarked = np.nonzero(grid == 0)
        lowest_cost = np.nonzero(path_length[unmarked] == np.nanmin(path_length[unmarked]))
        # next cell
        init = [unmarked[0][lowest_cost][0], unmarked[1][lowest_cost][0]]
    except:
        return "fail"

    if verbose:
        print(f"GRID:\n{grid}")
        print(f"PATH COST:\n{path_length}")

    # run the next iteration with the new cell
    return search(grid, init, goal, cost, path_length=path_length, verbose=verbose)


In [19]:
out = search(grid, init, goal, cost, verbose=True)

GRID:
[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [ 2. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 1 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [ 2.  3. nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
GRID:
[[1 1 1 0 0 0]
 [1 1 1 0 0 0]
 [1 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PA

In [5]:
out

[11, 4, 5]

## Expansion grid
Add an array `expand` tp the above search code that shows for each cell at which step they were expanded. Cells that were never expanded to should be marked -1. All other numbers should be unique.  
Return the `expand` array.  
My solution above passes over some cells multiple times, so this must be changed.

In [49]:
def search(grid, init, goal, cost, verbose=False, path_length=None, expand=None, _iter=0):
    if init == goal:
        return expand

    if not isinstance(grid, np.ndarray):
        grid = np.array(grid)
    else:
        grid = grid.copy()

    delta = np.array([
        [-1, 0], # go up
        [ 0,-1], # go left
        [ 1, 0], # go down
        [ 0, 1] # go right
        ])

    # path_length: initialise as nan
    if path_length is None:
        path_length = np.zeros(grid.shape)
        path_length[:, :] = np.nan
        # add initial cell with path length 0
        path_length[init[0], init[1]] = 0

    # expansion array: initialise as -1
    if expand is None:
        expand = np.ones(grid.shape) * -1
        # initial cell at expansion 0
        expand[init[0], init[1]] = 0
        _iter += 1

    # mark off the current cell
    grid[init[0], init[1]] = 1

    # check whether neighbouring cells are unmarked (untraveled)
    for d in delta:
        index = init - d
        try:
            # ignore negative indices
            if any(index < 0):
                continue

            cell = grid[index[0], index[1]]
            path = path_length[index[0], index[1]]
        except(IndexError):
            # ignore out of bounds indices
            continue

        # if cell is unmarked, update the path cost
        if (cell == 0) and (np.isnan(path)):
            path_length[index[0], index[1]] = path_length[init[0], init[1]] + cost
            # keep track of the iteration in which the cell was expanded
            expand[index[0], index[1]] = _iter
            _iter += 1

    try:
        # find the next cell: unmarked cell with lowest cost
        unmarked = np.nonzero(grid == 0)
        lowest_cost = np.nonzero(path_length[unmarked] == np.nanmin(path_length[unmarked]))
        # next cell
        init = [unmarked[0][lowest_cost][0], unmarked[1][lowest_cost][0]]
    except:
        return expand

    if verbose:
        print(f"GRID:\n{grid}")
        print(f"PATH COST:\n{path_length}")
        print(f"EXPAND:\n{expand}")

    # run the next iteration with the new cell
    return search(
        grid, init, goal, cost, path_length=path_length,
        expand=expand, verbose=verbose, _iter=_iter
        )


In [50]:
grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

out = search(grid, init, goal, cost, verbose=True)

GRID:
[[1 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
EXPAND:
[[ 0.  2. -1. -1. -1. -1.]
 [ 1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]]
GRID:
[[1 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
EXPAND:
[[ 0.  2. -1. -1. -1. -1.]
 [ 1.  3. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1.]]
GRID:
[[1 1 1 0 0 0]
 [1 0 1 0 0 0]
 [0 0 0 0 1 0]
 [0 0 1 1 1 0]
 [0 0 0 0 1 0]]
PATH COST:
[[ 0.  1. nan nan nan nan]
 [ 1.  2. nan nan nan nan]
 [ 2. nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
EXPAND:
[[ 0.  2. -1. -1.

In [65]:
out

array([[ 0.,  2., -1., 15., 17., 19.],
       [ 1.,  3., -1., 12., 14., 18.],
       [ 4.,  5.,  8., 10., -1., 20.],
       [ 6.,  7., -1., -1., -1., 21.],
       [ 9., 11., 13., 16., -1., 22.]])

## Print path
Modify the the search function so that it returns
a shortest path as follows:
```
[['>', 'v', ' ', ' ', ' ', ' '],
 [' ', '>', '>', '>', '>', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]
```
Where '>', '<', '^', and 'v' refer to right, left, 
up, and down motions. Note that the 'v' should be 
lowercase. '*' should mark the goal cell.

You may assume that all test cases for this function
will have a path from init to goal.

In [12]:
grid = [[0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 0]]
init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

def search(grid,init,goal,cost):
    # ----------------------------------------
    # modify code below
    # ----------------------------------------
    closed = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
    closed[init[0]][init[1]] = 1

    x = init[0]
    y = init[1]
    g = 0

    open = [[g, x, y]]

    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    expand[x][y] = 0

    found = False  # flag that is set when search is complete
    resign = False # flag set if we can't find expand

    count = 0

    while not found and not resign:
        if len(open) == 0:
            resign = True
            return 'fail'
        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            g = next[0]
            if x == goal[0] and y == goal[1]:
                found = True
            else:
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >=0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            count += 1
                            g2 = g + cost
                            open.append([g2, x2, y2])
                            closed[x2][y2] = 1
                            expand[x2][y2] = count

    # find the shortest path by stepping backwards from the goal
    path = [[' ' for col in range(len(grid[0]))] for row in range(len(grid))]
    path[goal[0]][goal[1]] = '*'
    x = goal[0]
    y = goal[1]
    path_found = False

    # path direction strings reordered for backwards path
    d_name = ['v', '>', '^', '<']

    while not path_found:
        if x == init[0] and y == init[1]:
            path_found = True
            return path
        
        previous = []

        for d, name in zip(delta, d_name):
            x2 = x + d[0]
            y2 = y + d[1]

            # check boundaries
            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                if expand[x2][y2] != -1:
                    previous.append([expand[x2][y2], x2, y2, name])

        previous.sort()
        previous.reverse()
        prev_cell = previous.pop()
        x = prev_cell[1]
        y = prev_cell[2]
        path[x][y] = prev_cell[3]


In [13]:
search(grid, init, goal, cost)

[['v', ' ', ' ', ' ', ' ', ' '],
 ['>', '>', '>', '>', '>', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]

## A*
In A*, you use a heuristic to determine the direction in which to expand. This leads to fewer expansions in the case of a grid with obstacles.  
Heuristic matrix H gives a heuristic of the number of steps to the goal from each cell, without obstacles.  
H = 
```
[[9, 8, 7, 6, 5, 4],
[8, 7, 6, 5, 4, 3],
[7, 6, 5, 4, 3, 2],
[6, 5, 4, 3, 2, 1],
[5, 4, 3, 2, 1, 0]]
```
Instead of ordering by g (cost), you order by f:
```
f(x, y) = g + H(x, y)
```

As a result, you won't expand into cells that take you further away from the goal, unless you are forced to do so by an obstacle.

In [14]:
# -----------
# User Instructions:
#
# Modify the the search function so that it becomes
# an A* search algorithm as defined in the previous
# lectures.
#
# Your function should return the expanded grid
# which shows, for each element, the count when
# it was expanded or -1 if the element was never expanded.
# 
# If there is no path from init to goal,
# the function should return the string 'fail'
# ----------

grid = [[0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0]]
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

def search(grid,init,goal,cost,heuristic):
    # ----------------------------------------
    # modify the code below
    # ----------------------------------------
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[init[0]][init[1]] = 1

    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    action = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]

    x = init[0]
    y = init[1]
    g = 0
    f = g + heuristic[x][y]

    open = [[f, g, x, y]]

    found = False  # flag that is set when search is complete
    resign = False # flag set if we can't find expand
    count = 0
    
    while not found and not resign:
        if len(open) == 0:
            resign = True
            return "Fail"
        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[2]
            y = next[3]
            g = next[1]
            expand[x][y] = count
            count += 1
            
            if x == goal[0] and y == goal[1]:
                found = True
            else:
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >=0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            f = g2 + heuristic[x2][y2]
                            open.append([f, g2, x2, y2])
                            closed[x2][y2] = 1

    return expand


In [15]:
search(grid, init, goal, cost, heuristic)

[[0, -1, -1, -1, -1, -1],
 [1, -1, -1, -1, -1, -1],
 [2, -1, -1, -1, -1, -1],
 [3, -1, 8, 9, 10, 11],
 [4, 5, 6, 7, -1, 12]]